# Part 1 - Stage 1 Data Manipulation

The purpose of this notebook is to analyze the stage 1 data and convert it into a csv file that can be loaded in model expoloration.

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pkg_resources

from binaryTree import Node
from PIL import Image, ImageDraw

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [3]:
cwd = os.getcwd()

## Load Data

In [4]:
tourney_cresults = pd.read_csv(cwd + '/data_stage2/MNCAATourneyCompactResults.csv')
seeds = pd.read_csv(cwd + '/data_stage2/MNCAATourneySeeds.csv')
seeds['Seed'] =  pd.to_numeric(seeds['Seed'].str[1:3], downcast='integer',errors='coerce')
season_dresults = pd.read_csv(cwd +'/data_stage2/MRegularSeasonDetailedResults.csv')

In [5]:
tourney_cresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [6]:
seeds.head()

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374


In [7]:
season_dresults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


## Analysis

Only analyze tournament games post 2003. There was a change to the tournament format in this year, so data before this is not as useful.

In [8]:
targetYear = 2003
tourney_cresults = tourney_cresults.loc[tourney_cresults['Season'] >= targetYear]

Initialize a dataframe for the training set

In [9]:
training_set = pd.DataFrame()

Add base features to training set

In [10]:
training_set['Result'] = np.random.randint(0,2,len(tourney_cresults.index))
training_set['Season'] = tourney_cresults['Season'].values
training_set['Team1'] = training_set['Result'].values * tourney_cresults['WTeamID'].values + (1-training_set['Result'].values) * tourney_cresults['LTeamID'].values 
training_set['Team2'] = (1-training_set['Result'].values) * tourney_cresults['WTeamID'].values + training_set['Result'].values * tourney_cresults['LTeamID'].values
training_set.head()

,Result,Season,Team1,Team2
0,1,2003,1421,1411
1,1,2003,1112,1436
2,1,2003,1113,1272
3,0,2003,1166,1141
4,0,2003,1301,1143


Define helper functions for populating the rest of the data in the training set.

In [11]:
def delta_seed(row):
    cond = (seeds['Season'] == row['Season'])
    return seeds[cond & (seeds['TeamID'] == row['Team1'])]['Seed'].iloc[0] - seeds[cond & (seeds['TeamID'] == row['Team2'])]['Seed'].iloc[0]

def delta_winPct(row):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1]['wins']/record[cond1]['games']).mean() - (record[cond2]['wins']/record[cond2]['games']).mean()

def get_points_against(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'LScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'WScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_points_for(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'WScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'LScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_remaining_stats(row, field):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'W' + field
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'L'+ field
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum()
    return retVal

def delta_stat(row, field):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1][field]/record[cond1]['games']).mean() - (record[cond2][field]/record[cond2]['games']).mean()
  

In [12]:
training_set['deltaSeed'] = training_set.apply(delta_seed,axis=1)
training_set.head()

,Result,Season,Team1,Team2,deltaSeed
0,1,2003,1421,1411,0
1,1,2003,1112,1436,-15
2,1,2003,1113,1272,3
3,0,2003,1166,1141,-5
4,0,2003,1301,1143,1


In [13]:
record = pd.DataFrame({'wins': season_dresults.groupby(['Season','WTeamID']).size()}).reset_index();
losses = pd.DataFrame({'losses': season_dresults.groupby(['Season','LTeamID']).size()}).reset_index();

record = record.merge(losses, how='outer', left_on=['Season','WTeamID'], right_on=['Season','LTeamID'])
record = record.fillna(0)
record['games'] = record['wins'] + record['losses']

In [14]:
# create dataframes of both winners and losers
dfW = season_dresults.groupby(['Season','WTeamID']).sum().reset_index()
dfL = season_dresults.groupby(['Season','LTeamID']).sum().reset_index()

In [15]:
# add points for and points against data
record['PointsFor'] = record.apply(get_points_for, axis=1)
record['PointsAgainst'] = record.apply(get_points_against, axis=1)

In [16]:
# This cell takes ~3 min. To slides
cols = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

for col in cols:
    print("Processing",col)
    record[col] = record.apply(get_remaining_stats, args=(col,), axis=1)

Processing FGM
Processing FGA
Processing FGM3
Processing FGA3
Processing FTM
Processing FTA
Processing OR
Processing DR
Processing Ast
Processing TO
Processing Stl
Processing Blk
Processing PF


You can calculate 

In [20]:
record['FGprct'] = record['FGM'] / record['FGA']  
record.head()

,Season,WTeamID,wins,LTeamID,losses,games,PointsFor,PointsAgainst,FGM,FGA,...,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,FGprct
0,2003,1102.0,12.0,1102.0,16.0,28.0,1603,1596,536,1114,...,312,479,117,471,364,320,167,50,525,0.481149
1,2003,1103.0,13.0,1103.0,14.0,27.0,2127,2110,733,1508,...,514,698,264,538,411,341,196,63,536,0.486074
2,2003,1104.0,17.0,1104.0,11.0,28.0,1940,1820,673,1601,...,416,586,380,670,339,372,185,106,505,0.420362
3,2003,1105.0,7.0,1105.0,19.0,26.0,1866,1993,634,1602,...,401,568,351,601,378,485,242,54,526,0.395755
4,2003,1106.0,13.0,1106.0,15.0,28.0,1781,1785,656,1548,...,298,461,344,668,327,477,234,88,509,0.423773


In [21]:
# This will take ~ 3 min. To slides
cols = ['PointsFor','PointsAgainst','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

for col in cols:
    print("Processing",col)
    training_set['delta' + col] = training_set.apply(delta_stat,args=(col,),axis=1)

Processing PointsFor
Processing PointsAgainst
Processing FGM
Processing FGA
Processing FGM3
Processing FGA3
Processing FTM
Processing FTA
Processing OR
Processing DR
Processing Ast
Processing TO
Processing Stl
Processing Blk
Processing PF


In [22]:
training_set['deltaWinPct'] = training_set.apply(delta_winPct,axis=1)
training_set.head()

,Result,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,deltaFGA,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,1,2003,1421,1411,0,-0.151724,-1.593103,7.614943,-0.354023,1.526437,...,-0.500000,-1.434483,-7.135632,-0.890805,-1.627586,-1.165517,0.973563,0.635632,0.766667,0.803448
1,1,2003,1112,1436,-15,0.237685,17.421182,7.112069,5.493842,9.852217,...,4.588670,4.673645,5.448276,2.213054,1.918719,3.435961,0.716749,1.602217,1.248768,1.853448
2,1,2003,1113,1272,3,-0.172414,1.448276,3.344828,0.931034,-3.103448,...,-7.482759,2.586207,3.310345,-0.379310,-2.655172,-1.068966,0.206897,-2.172414,-0.827586,0.655172
3,0,2003,1166,1141,-5,0.085684,-0.102403,-8.908046,2.076280,4.764890,...,2.553814,-5.397074,-5.142111,0.292581,-0.094044,1.197492,-4.877743,1.290491,0.454545,-3.692790
4,0,2003,1301,1143,1,-0.124138,-2.082759,-1.758621,-3.011494,-5.390805,...,5.465517,2.387356,0.949425,-1.508046,-2.345977,-1.333333,0.027586,1.214943,0.273563,1.563218


In [23]:
training_set.describe()

,Result,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,deltaFGA,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
count,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,...,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000
mean,0.514798,2011.096861,1297.506726,1292.327354,-0.223318,-0.001257,-0.158022,-0.170055,-0.050797,-0.052541,...,0.040384,-0.076020,-0.138715,-0.003730,-0.095215,0.074075,-0.016878,0.027197,-0.029744,0.055190
std,0.500005,4.896921,103.769522,104.613933,7.461174,0.144490,7.292109,6.406372,2.914471,4.878647,...,4.197347,2.707469,3.656976,2.562622,2.562278,2.597132,2.070296,1.851795,1.855861,2.532732
min,0.000000,2003.000000,1101.000000,1102.000000,-15.000000,-0.633333,-21.643382,-18.356855,-8.602941,-17.404412,...,-14.278495,-7.569697,-10.827586,-7.924632,-9.010776,-8.363636,-7.666667,-5.352941,-6.752688,-7.896552
25%,0.000000,2007.000000,1214.000000,1207.500000,-5.000000,-0.093880,-5.177557,-4.651042,-2.023920,-3.403743,...,-2.671176,-1.925335,-2.603030,-1.663033,-1.752367,-1.694129,-1.425433,-1.114240,-1.190524,-1.719697
50%,1.000000,2011.000000,1292.000000,1281.000000,0.000000,-0.000949,-0.275202,-0.319244,-0.093750,0.001894,...,0.136364,-0.133690,-0.074811,0.048295,-0.103387,-0.036932,0.016934,-0.011089,-0.017992,0.000000
75%,1.000000,2015.000000,1393.000000,1393.000000,5.000000,0.089957,4.437556,4.007148,1.956345,3.120968,...,2.767268,1.742800,2.171225,1.719981,1.568365,1.805675,1.252367,1.278848,1.154703,1.821023
max,1.000000,2019.000000,1462.000000,1463.000000,15.000000,0.491657,22.892857,25.035714,9.357143,22.071429,...,13.394831,8.059629,10.719453,9.214286,9.285714,9.220143,7.580645,6.283681,6.242424,7.230303


In [24]:
training_set.to_csv("training_set.csv", index=False)
record.to_csv("record.csv", index=False)

In [26]:
training_set[training_set['Season']==2019]

,Result,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,deltaFGA,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
1048,0,2019,1396,1125,0,-0.114583,-12.022917,-3.510417,-5.260417,-3.514583,...,-5.279167,1.364583,1.975000,0.818750,-5.104167,-5.131250,-0.443750,1.989583,-1.550000,1.768750
1049,0,2019,1341,1192,0,0.055718,0.106549,0.243402,-0.678397,3.380254,...,0.344086,3.087977,6.330401,1.986315,-1.127077,-1.632454,-0.845552,1.302053,-2.039101,4.955034
1050,0,2019,1385,1113,0,-0.051136,-0.297348,1.725379,1.345644,2.076705,...,2.325758,-4.253788,-7.815341,-4.519886,-2.903409,0.436553,-3.230114,2.599432,0.114583,-0.845644
1051,0,2019,1300,1295,0,-0.016129,-4.590323,-5.875269,-0.313978,0.223656,...,-6.254839,-0.519355,0.784946,4.952688,0.739785,3.110753,4.854839,0.524731,0.082796,2.904301
1052,1,2019,1120,1308,-7,-0.139706,1.226103,3.775735,-0.314338,1.159926,...,2.439338,-0.181985,-1.452206,-0.889706,-3.183824,-0.088235,0.115809,3.762868,2.295956,0.288603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,1,2019,1120,1246,3,-0.082888,2.155080,3.163993,0.423351,4.835116,...,13.394831,-4.073084,-4.901070,0.129234,-4.877005,0.563280,-0.489305,3.263815,-0.295900,2.079323
1111,1,2019,1277,1181,1,-0.029412,-4.676471,-2.029412,-2.735294,-6.735294,...,-2.441176,-0.235294,-2.088235,-2.558824,1.705882,3.029412,-0.264706,-4.235294,-1.352941,1.058824
1112,1,2019,1403,1277,1,-0.011029,-5.729779,-6.248162,-1.608456,-1.707721,...,-1.898897,-1.468750,-1.406250,-1.979779,-4.650735,-4.909926,-0.477941,2.139706,-0.564338,0.869485
1113,0,2019,1120,1438,4,-0.170956,7.038603,13.525735,1.404412,6.847426,...,9.251838,1.286765,2.485294,2.860294,-3.808824,-0.088235,3.115809,3.669118,0.952206,3.726103
